In [1]:
# Deep Convolutional GANs

# Importing the libraries
# from __future__ import print_function


import os
import random
import torch.nn.parallel
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

import tenseal as ts
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
from mpi4py import MPI


In [2]:
th.manual_seed(73)
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

In [3]:
# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

In [4]:
# Creating the transformations
transform = transforms.Compose([transforms.Resize(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.
nc = 3

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
train_X = th.load("data-2/train_X.pt")
train_y = th.load("data-2/train_y.pt")
test_X = th.load("data-2/test_X.pt")
test_y = th.load("data-2/test_y.pt")
#clinical_data = torch.stack(clinical_data,dim=1).squeeze(0) #(torch.arange(clinical_data), dim=1)

In [7]:
# Training dataset
train_dataset = TensorDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, batch_size=64, num_workers=2)
# Test dataset
test_dataset = TensorDataset(test_X, test_y)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=2)

In [8]:
def shuffleDiscriminators():
    if (rank != 0):
        layer_num = 0
        for param in model.parameters():
            outdata = param.data.numpy().copy()
            indata = None

            if (rank != size - 1):
                comm.send(outdata, dest=rank + 1, tag=1)
            if (rank != 1):
                indata = comm.recv(source = rank-1, tag=1)

            if (rank == size - 1):
                comm.send(outdata, dest=1, tag=2)
            if (rank == 1):
                indata = comm.recv(source = size - 1, tag=2)
            # Shuffling the Discriminator
            param.data = torch.from_numpy(indata)
            layer_num += 1

In [9]:
class Model(nn.Module):
    def __init__(self): 
        super(Model, self).__init__()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 12)
        
    def forward(self, x):
        out = self.fc1(x)
        out = out * out
        out = self.fc2(out)
        return out

In [10]:
def train(model, device, train_loader, optimizer, criterion, epochs):
    losses = []
    for epoch in range(1, epochs + 1):
        model.train()
        shuffleDiscriminators()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        model.eval()
        print('Train Epoch: {:2d}   Avg Loss: {:.6f}'.format(epoch, th.mean(th.tensor(losses))))

    return model

In [19]:
model = Model()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
device = th.device("cuda" if th.cuda.is_available() else "cpu")

device = th.device("cpu")

model = train(model, device, train_loader, optimizer, criterion, 30)

Train Epoch:  1   Avg Loss: 0.668271
Train Epoch:  2   Avg Loss: 0.610264
Train Epoch:  3   Avg Loss: 0.541148
Train Epoch:  4   Avg Loss: 0.492275
Train Epoch:  5   Avg Loss: 0.458228
Train Epoch:  6   Avg Loss: 0.433085
Train Epoch:  7   Avg Loss: 0.413479
Train Epoch:  8   Avg Loss: 0.397509
Train Epoch:  9   Avg Loss: 0.384047
Train Epoch: 10   Avg Loss: 0.372386
Train Epoch: 11   Avg Loss: 0.362065
Train Epoch: 12   Avg Loss: 0.352769
Train Epoch: 13   Avg Loss: 0.344287
Train Epoch: 14   Avg Loss: 0.336466
Train Epoch: 15   Avg Loss: 0.329201
Train Epoch: 16   Avg Loss: 0.322416
Train Epoch: 17   Avg Loss: 0.316050
Train Epoch: 18   Avg Loss: 0.310059
Train Epoch: 19   Avg Loss: 0.304405
Train Epoch: 20   Avg Loss: 0.299055
Train Epoch: 21   Avg Loss: 0.293985
Train Epoch: 22   Avg Loss: 0.289173
Train Epoch: 23   Avg Loss: 0.284598
Train Epoch: 24   Avg Loss: 0.280242
Train Epoch: 25   Avg Loss: 0.276089
Train Epoch: 26   Avg Loss: 0.272124
Train Epoch: 27   Avg Loss: 0.268333
T

In [20]:
def compute_labels(out):
    out = th.sigmoid(out)
    return (out >= 0.5).int()


# compute accuracy using hamming loss
def accuracy(output, target):
    # convert to labels
    out = compute_labels(output)
    # flatten and compute hamming loss
    flat_out = out.flatten()
    flat_target = target.flatten()
    incorrect = th.logical_xor(flat_out, flat_target).sum().item()
    hamming_loss = incorrect / len(flat_out)
    return 1 - hamming_loss


print("Accuracy on test set: {:.2f}".format(accuracy(model(test_X), test_y)))

Accuracy on test set: 0.92


Below is for encrypted model using PyTorch-like model, but which uses TenSEAL operations.

In [21]:
class HEModel:
    def __init__(self, fc1, fc2):
        self.fc1_weight = fc1.weight.t().tolist()
        self.fc1_bias = fc1.bias.tolist()
        self.fc2_weight = fc2.weight.t().tolist()
        self.fc2_bias = fc2.bias.tolist()
        
    def forward(self, encrypted_vec):
        # first fc layer + square activation function
        encrypted_vec = encrypted_vec.mm(self.fc1_weight) + self.fc1_bias
        encrypted_vec *= encrypted_vec
        # second fc layer
        encrypted_vec = encrypted_vec.mm(self.fc2_weight) + self.fc2_bias
        return encrypted_vec
    
    def __call__(self, x):
        return self.forward(x)

In [22]:
def shuffleHEDiscriminators():
    if (rank != 0):
        layer_num = 0
        for param in he_model.parameters():
            outdata = param.data.numpy().copy()
            indata = None

            if (rank != size - 1):
                comm.send(outdata, dest=rank + 1, tag=1)
            if (rank != 1):
                indata = comm.recv(source = rank-1, tag=1)

            if (rank == size - 1):
                comm.send(outdata, dest=1, tag=2)
            if (rank == 1):
                indata = comm.recv(source = size - 1, tag=2)
                # Shuffling the Discriminator
            param.data = torch.from_numpy(indata)
            layer_num += 1

In [32]:
# Parameters
bits_scale = 25
coeff_mod_bit_sizes = [30, bits_scale, bits_scale, bits_scale, 30]
polynomial_modulus_degree = 8192

# Create context
context = ts.context(ts.SCHEME_TYPE.CKKS, polynomial_modulus_degree, coeff_mod_bit_sizes=coeff_mod_bit_sizes)
# Set global scale
context.global_scale = 2 ** bits_scale
# Generate galois keys required for matmul in ckks_vector
context.generate_galois_keys()

he_model = HEModel(model.fc1, model.fc2)

Below is Encrypted evaluation of entire dataset.
Steps:
1. encrypt the vector
2. do encrypted evaluation
3. decrypt the result



In [24]:
def he_train(he_model, device, train_loader, criterion, optimizer, epochs):
    losses = []
    for epoch in range(1, epochs + 1):
        model.train()
        shuffleHEDiscriminators()
        for batch_idx, (data, target) in enumerate(train_loader):
            #data, target = data.to(device), target.to(device)
            #clinical_data = torch.stack(clinical_data,dim=1).squeeze(0) #(torch.arange(clinical_data), dim=1)
            vec_data = data.flatten()
            vec_data = vec_data.flatten()
            vec_target = data.flatten()
            
            encrypted_vec_data = ts.ckks_vector(context, vec_data)
            encrypted_vec_target = ts.ckks_vector(context, vec_target)
            
            encrypted_out_data = he_model(encrypted_vec_data)
            encrypted_out_target = he_model(encrypted_vec_target)
            
            optimizer.zero_grad()
            output = he_model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        model.eval()
        print('Train Epoch: {:2d}   Avg Loss: {:.6f}'.format(epoch, th.mean(th.tensor(losses))))

    return model

In [30]:
he_train(model, device, train_loader, optimizer, criterion, 30)

The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.


ValueError: can't execute matmul_plain on chunked vectors

In [ ]:
# how many labels in the encrypted evaluation are the same as in the plain evaluation?
match = 0
he_outs = []
for data, _ in test_loader:
    # remove batch axis, we only need a flat vector
    vec = data.flatten()
    # encryption
    encrypted_vec = ts.ckks_vector(context, vec)
    # encrypted evaluation
    encrypted_out = he_model(encrypted_vec)
    # decryptionhe_epoch = 1
    he_out = th.tensor(encrypted_out.decrypt())
    he_outs.append(he_out.tolist())
    out = model(data)
    # how many labels match
    he_labels = compute_labels(he_out)
    plain_labels = compute_labels(out)
    match += (he_labels == plain_labels).sum().item()

In [ ]:
print("Accuracy on test set (encrypted evaluation): {:.2f}".format(accuracy(th.tensor(he_outs), test_y)))
print("Encrypted evaluation matched {:.1f}% of the labels from the plain evaluation".format(
    match / (12 * len(test_loader)) * 100)
)

For 2 party implementation

-send the encrypted vector for remote evaluation

-then send back encrypted resul for decryption